In [ ]:
%%bash
# Get data from https://drive.google.com/file/d/1HuxIvMWF-0xbgza1Y_o5vYVwtpv-fekZ/view?usp=sharing
cat Data/*.txt > Data/consolidated_data.csv

In [ ]:
import pandas as pd 
raw = pd.read_csv('Data/consolidated_data.csv')
raw.columns = ['time','lat','long','mobile','wifi']
raw = raw.drop_duplicates().reset_index(drop=True)
raw

In [ ]:
# cleanup wifi data
raw.wifi = raw.wifi.fillna(value=-127)

In [ ]:
# cleanup mobile data
raw.mobile = raw.mobile.fillna(value=-127)

In [ ]:
# positive offsetting for ease of data manipulation
raw.wifi = raw.wifi.add(127)
raw.mobile = raw.mobile.add(127)

from datetime import datetime

def date_string_to_epoch(date_string):
    dt_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')
    epoch_timestamp = int(dt_object.timestamp())
    return epoch_timestamp

raw.time = raw.time.apply(date_string_to_epoch)
raw



In [ ]:
# splitting into wifi and mobile data
wifi = raw[raw.wifi > 0].reset_index(drop=True).rename(columns={'wifi':'wifi_strength'})
wifi = wifi.drop(columns='mobile')
mobile = raw[raw.mobile > 0].reset_index(drop=True).rename(columns={'mobile':'mobile_strength'})
mobile = mobile.drop(columns='wifi')

In [ ]:
wifi

In [ ]:
mobile

In [ ]:
mobile.to_csv('Data/mobile_cleaned.csv',index=False)
wifi.to_csv('Data/wifi_cleaned.csv',index=False)